<a href="https://colab.research.google.com/github/sumankanukollu/denseDepthMaps_S15/blob/master/EVA4_S15A_DenseDepth_400K.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1.  Mount drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


# 2. Install auto-time

In [3]:
!pip install ipython-autotime
%load_ext autotime

  Created wheel for ipython-autotime: filename=ipython_autotime-0.1-cp36-none-any.whl size=1832 sha256=ccadc594f7cfcf158fbd3ecccc3bf45b9550509e47858af8666785bd0bea9389
  Stored in directory: /root/.cache/pip/wheels/d2/df/81/2db1e54bc91002cec40334629bc39cfa86dff540b304ebcd6e
Successfully built ipython-autotime


# 3. Hardware details

In [1]:
import torch
print('PyTorch %s %s' % (torch.__version__, torch.cuda.get_device_properties(0) if torch.cuda.is_available() else 'CPU'))

PyTorch 1.5.0+cu101 _CudaDeviceProperties(name='Tesla P100-PCIE-16GB', major=6, minor=0, total_memory=16280MB, multi_processor_count=56)


In [13]:
#!rm -rf denseDepthMaps_S15

time: 5.47 s


# 4. Clone our own Git hub repo 

In [0]:
import os
#os.chdir('/content/drive/My Drive/EVA4/S15A_RCNN_RGB')
#!git pull
#!git clone https://github.com/sumankanukollu/denseDepthMaps_S15.git

In [113]:
os.chdir('/content/drive/My Drive/EVA4/S15A_RCNN_RGB/denseDepthMaps_S15')
!git pull

remote: Enumerating objects: 7, done.
remote: Counting objects: 100% (7/7), done.
remote: Compressing objects: 100% (2/2), done.
remote: Total 4 (delta 2), reused 4 (delta 2), pack-reused 0
Unpacking objects: 100% (4/4), done.
From https://github.com/sumankanukollu/denseDepthMaps_S15
   f71ef15..efa45a6  master     -> origin/master
Updating f71ef15..efa45a6
Fast-forward
 DenseDepth/utils.py | 1 +
 1 file changed, 1 insertion(+)
time: 4.09 s


# 5. Download the pre-trained model

In [15]:
!ls
import os
os.chdir('/content/drive/My Drive/EVA4/S15A_RCNN_RGB/denseDepthMaps_S15')
!wget https://s3-eu-west-1.amazonaws.com/densedepth/nyu.h5 -O ./DenseDepth/nyu.h5

bg_fg_mask_jpgIms	       EVA4_S15A_Suman_800K.ipynb
datasetPreparationSteps.ipynb  extract_BG_FG_Images.ipynb
DenseDepth.ipynb	       original_png_imgs
denseDepthMaps_S15	       resize.py
depthMaps_segmentation	       semantic_segmentation_on_Image_800K.ipynb
EVA4_S15A_DenseDepth.ipynb
--2020-05-10 05:34:24--  https://s3-eu-west-1.amazonaws.com/densedepth/nyu.h5
Resolving s3-eu-west-1.amazonaws.com (s3-eu-west-1.amazonaws.com)... 52.218.53.226
Connecting to s3-eu-west-1.amazonaws.com (s3-eu-west-1.amazonaws.com)|52.218.53.226|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 172897376 (165M) [application/h5]
Saving to: ‘./DenseDepth/nyu.h5’

./DenseDepth/nyu.h5 100%[===================>] 164.89M  22.6MB/s    in 8.0s    

2020-05-10 05:34:33 (20.7 MB/s) - ‘./DenseDepth/nyu.h5’ saved [172897376/172897376]

time: 13.2 s


# 6. Change the directories and modified files like handy if needed



In [25]:
os.chdir('/content/drive/My Drive/EVA4/S15A_RCNN_RGB/denseDepthMaps_S15/DenseDepth')
!ls

augment.py	demo_rgb.npy	      layers_bkp.py  nyu.h5	  test_bkp.py
callbacks.py	DenseDepth_bkp.ipynb  layers.py      __pycache__  test.py
data.py		DenseDepth.ipynb      LICENSE	     PyTorch	  train.py
demo_depth.npy	evaluate.py	      loss.py	     README.md	  utils_bkp.py
demo.py		examples	      model.py	     Tensorflow   utils.py
time: 2.83 s


In [0]:
import os,gc
from google.colab import files
from zipfile import ZipFile
from PIL import Image
#!ls
os.chdir('/content/drive/My Drive/EVA4/S15A_RCNN_RGB/denseDepthMaps_S15/DenseDepth')

In [0]:
#os.remove('layers.py')
#src = list(files.upload().values())[0]
#open('layers.py','wb').write(src)

In [0]:
#os.remove('utils.py')
#src = list(files.upload().values())[0]
#open('utils.py','wb').write(src)

In [0]:
#os.remove('test.py')
#src = list(files.upload().values())[0]
#open('test.py','wb').write(src)


# Clear garbage and verify the number of depth map images generated, after each batch

In [0]:
def verifyCount():
    gc.enable()
    gc.collect()
    print('Total number of depthmap images generated as of now : {} '.format(len(os.listdir('/content/drive/My Drive/EVA4/S15A_RCNN_RGB/denseDepthMaps_S15/DenseDepth/depthMapOutput'))))
verifyCount()

# 7. Start execution in batches of 10K each batch

In [114]:
os.chdir('/content/drive/My Drive/EVA4/S15A_RCNN_RGB/denseDepthMaps_S15/DenseDepth')

time: 2.15 ms


## output_4.zip : batches

### Batches : 1

In [0]:
!python test.py --input '/content/drive/My Drive/EVA4/S15A_RCNN_RGB/bg_fg_mask_jpgIms/output_4.zip' --start 0 --end 5000 -o '/content/drive/My Drive/EVA4/S15A_RCNN_RGB/denseDepthMaps_S15/DenseDepth/depthMapOutput' -rs 0 -re 10001

### Batches : 2

In [0]:
!python test.py --input '/content/drive/My Drive/EVA4/S15A_RCNN_RGB/bg_fg_mask_jpgIms/output_4.zip' --start 0 --end 5000 -o '/content/drive/My Drive/EVA4/S15A_RCNN_RGB/denseDepthMaps_S15/DenseDepth/depthMapOutput' -rs 10000 -re 20003

### Batches : 3

In [52]:
!python test.py --input '/content/drive/My Drive/EVA4/S15A_RCNN_RGB/bg_fg_mask_jpgIms/output_4.zip' --start 0 --end 5000 -o '/content/drive/My Drive/EVA4/S15A_RCNN_RGB/denseDepthMaps_S15/DenseDepth/depthMapOutput' -rs 20000 -re 30001

Using TensorFlow backend.
Loading model...

Model loaded (nyu.h5).
tcmalloc: large alloc 6021120000 bytes == 0x7f3845c10000 @  0x7f3aa20491e7 0x7f3a9ea3e5e1 0x7f3a9eaa2c78 0x7f3a9eaa2d93 0x7f3a9eb40ea8 0x7f3a9eb41704 0x7f3a9eb41852 0x566d63 0x59fc4e 0x7f3a9ea8e4ed 0x50a2bf 0x50bfb4 0x507d64 0x509a90 0x50a48d 0x50cd96 0x507d64 0x588e5c 0x59fc4e 0x7f3a9ea8e4ed 0x50a2bf 0x50bfb4 0x507d64 0x509a90 0x50a48d 0x50cd96 0x507d64 0x509a90 0x50a48d 0x50cd96 0x507d64

Loaded (5000) images of size (224, 224, 3).
tcmalloc: large alloc 6021120000 bytes == 0x7f34ed1d0000 @  0x7f3aa20491e7 0x7f3a9ea3e5e1 0x7f3a9eaa5420 0x7f3a9ea9a7d5 0x7f3a9eb2f497 0x50a635 0x50bfb4 0x507d64 0x50ae13 0x634c82 0x634d37 0x6384ef 0x639091 0x4b0d00 0x7f3aa1c46b97 0x5b250a
/content/drive/My Drive/EVA4/S15A_RCNN_RGB/denseDepthMaps_S15/DenseDepth/utils.py:128: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed an

In [53]:
print(len(os.listdir('/content/drive/My Drive/EVA4/S15A_RCNN_RGB/denseDepthMaps_S15/DenseDepth/depthMapOutput')))

15001
time: 154 ms


### Batches : 4

In [55]:
!python test.py --input '/content/drive/My Drive/EVA4/S15A_RCNN_RGB/bg_fg_mask_jpgIms/output_4.zip' --start 0 --end 5000 -o '/content/drive/My Drive/EVA4/S15A_RCNN_RGB/denseDepthMaps_S15/DenseDepth/depthMapOutput' -rs 30000 -re 40001

Using TensorFlow backend.
Loading model...

Model loaded (nyu.h5).
tcmalloc: large alloc 6021120000 bytes == 0x7f2f7a990000 @  0x7f31d6db51e7 0x7f31d37aa5e1 0x7f31d380ec78 0x7f31d380ed93 0x7f31d38acea8 0x7f31d38ad704 0x7f31d38ad852 0x566d63 0x59fc4e 0x7f31d37fa4ed 0x50a2bf 0x50bfb4 0x507d64 0x509a90 0x50a48d 0x50cd96 0x507d64 0x588e5c 0x59fc4e 0x7f31d37fa4ed 0x50a2bf 0x50bfb4 0x507d64 0x509a90 0x50a48d 0x50cd96 0x507d64 0x509a90 0x50a48d 0x50cd96 0x507d64

Loaded (5000) images of size (224, 224, 3).
tcmalloc: large alloc 6021120000 bytes == 0x7f2c151d0000 @  0x7f31d6db51e7 0x7f31d37aa5e1 0x7f31d3811420 0x7f31d38067d5 0x7f31d389b497 0x50a635 0x50bfb4 0x507d64 0x50ae13 0x634c82 0x634d37 0x6384ef 0x639091 0x4b0d00 0x7f31d69b2b97 0x5b250a
/content/drive/My Drive/EVA4/S15A_RCNN_RGB/denseDepthMaps_S15/DenseDepth/utils.py:128: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed an

In [56]:
print(len(os.listdir('/content/drive/My Drive/EVA4/S15A_RCNN_RGB/denseDepthMaps_S15/DenseDepth/depthMapOutput')))

20001
time: 224 ms


### Batches : 5

In [57]:
!python test.py --input '/content/drive/My Drive/EVA4/S15A_RCNN_RGB/bg_fg_mask_jpgIms/output_4.zip' --start 0 --end 5000 -o '/content/drive/My Drive/EVA4/S15A_RCNN_RGB/denseDepthMaps_S15/DenseDepth/depthMapOutput' -rs 40000 -re 50001


Using TensorFlow backend.
Loading model...

Model loaded (nyu.h5).
tcmalloc: large alloc 6021120000 bytes == 0x7feabf090000 @  0x7fed1b4af1e7 0x7fed17ea45e1 0x7fed17f08c78 0x7fed17f08d93 0x7fed17fa6ea8 0x7fed17fa7704 0x7fed17fa7852 0x566d63 0x59fc4e 0x7fed17ef44ed 0x50a2bf 0x50bfb4 0x507d64 0x509a90 0x50a48d 0x50cd96 0x507d64 0x588e5c 0x59fc4e 0x7fed17ef44ed 0x50a2bf 0x50bfb4 0x507d64 0x509a90 0x50a48d 0x50cd96 0x507d64 0x509a90 0x50a48d 0x50cd96 0x507d64

Loaded (5000) images of size (224, 224, 3).
tcmalloc: large alloc 6021120000 bytes == 0x7fe7671d0000 @  0x7fed1b4af1e7 0x7fed17ea45e1 0x7fed17f0b420 0x7fed17f007d5 0x7fed17f95497 0x50a635 0x50bfb4 0x507d64 0x50ae13 0x634c82 0x634d37 0x6384ef 0x639091 0x4b0d00 0x7fed1b0acb97 0x5b250a
/content/drive/My Drive/EVA4/S15A_RCNN_RGB/denseDepthMaps_S15/DenseDepth/utils.py:128: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed an

In [58]:
print(len(os.listdir('/content/drive/My Drive/EVA4/S15A_RCNN_RGB/denseDepthMaps_S15/DenseDepth/depthMapOutput')))

25001
time: 252 ms


### Batches : 6

In [61]:
!python test.py --input '/content/drive/My Drive/EVA4/S15A_RCNN_RGB/bg_fg_mask_jpgIms/output_4.zip' --start 0 --end 5000 -o '/content/drive/My Drive/EVA4/S15A_RCNN_RGB/denseDepthMaps_S15/DenseDepth/depthMapOutput' -rs 50000 -re 60001
verifyCount()

Using TensorFlow backend.
Loading model...

Model loaded (nyu.h5).
tcmalloc: large alloc 6021120000 bytes == 0x7f2269310000 @  0x7f24c57361e7 0x7f24c212b5e1 0x7f24c218fc78 0x7f24c218fd93 0x7f24c222dea8 0x7f24c222e704 0x7f24c222e852 0x566d63 0x59fc4e 0x7f24c217b4ed 0x50a2bf 0x50bfb4 0x507d64 0x509a90 0x50a48d 0x50cd96 0x507d64 0x588e5c 0x59fc4e 0x7f24c217b4ed 0x50a2bf 0x50bfb4 0x507d64 0x509a90 0x50a48d 0x50cd96 0x507d64 0x509a90 0x50a48d 0x50cd96 0x507d64

Loaded (5000) images of size (224, 224, 3).
tcmalloc: large alloc 6021120000 bytes == 0x7f1f031d0000 @  0x7f24c57361e7 0x7f24c212b5e1 0x7f24c2192420 0x7f24c21877d5 0x7f24c221c497 0x50a635 0x50bfb4 0x507d64 0x50ae13 0x634c82 0x634d37 0x6384ef 0x639091 0x4b0d00 0x7f24c5333b97 0x5b250a
/content/drive/My Drive/EVA4/S15A_RCNN_RGB/denseDepthMaps_S15/DenseDepth/utils.py:128: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed an

### Batches : 7

In [71]:
!python test.py --input '/content/drive/My Drive/EVA4/S15A_RCNN_RGB/bg_fg_mask_jpgIms/output_4.zip' --start 0 --end 5000 -o '/content/drive/My Drive/EVA4/S15A_RCNN_RGB/denseDepthMaps_S15/DenseDepth/depthMapOutput' -rs 60000 -re 70001
verifyCount()

Using TensorFlow backend.
Loading model...

Model loaded (nyu.h5).
tcmalloc: large alloc 6021120000 bytes == 0x7ff3baed0000 @  0x7ff6172ec1e7 0x7ff613ce15e1 0x7ff613d45c78 0x7ff613d45d93 0x7ff613de3ea8 0x7ff613de4704 0x7ff613de4852 0x566d63 0x59fc4e 0x7ff613d314ed 0x50a2bf 0x50bfb4 0x507d64 0x509a90 0x50a48d 0x50cd96 0x507d64 0x588e5c 0x59fc4e 0x7ff613d314ed 0x50a2bf 0x50bfb4 0x507d64 0x509a90 0x50a48d 0x50cd96 0x507d64 0x509a90 0x50a48d 0x50cd96 0x507d64

Loaded (5000) images of size (224, 224, 3).
tcmalloc: large alloc 6021120000 bytes == 0x7ff0631d0000 @  0x7ff6172ec1e7 0x7ff613ce15e1 0x7ff613d48420 0x7ff613d3d7d5 0x7ff613dd2497 0x50a635 0x50bfb4 0x507d64 0x50ae13 0x634c82 0x634d37 0x6384ef 0x639091 0x4b0d00 0x7ff616ee9b97 0x5b250a
500
1000
1500
2000
2500
3000
3500
4000
4500
5000
done
Total number of depthmap images generated as of now : 35001 
time: 9min 33s


### Batches : 8

In [72]:
!python test.py --input '/content/drive/My Drive/EVA4/S15A_RCNN_RGB/bg_fg_mask_jpgIms/output_4.zip' --start 0 --end 5000 -o '/content/drive/My Drive/EVA4/S15A_RCNN_RGB/denseDepthMaps_S15/DenseDepth/depthMapOutput' -rs 70000 -re 80001
verifyCount()

Using TensorFlow backend.
Loading model...

Model loaded (nyu.h5).
tcmalloc: large alloc 6021120000 bytes == 0x7f7a37b90000 @  0x7f7c93fbc1e7 0x7f7c909b15e1 0x7f7c90a15c78 0x7f7c90a15d93 0x7f7c90ab3ea8 0x7f7c90ab4704 0x7f7c90ab4852 0x566d63 0x59fc4e 0x7f7c90a014ed 0x50a2bf 0x50bfb4 0x507d64 0x509a90 0x50a48d 0x50cd96 0x507d64 0x588e5c 0x59fc4e 0x7f7c90a014ed 0x50a2bf 0x50bfb4 0x507d64 0x509a90 0x50a48d 0x50cd96 0x507d64 0x509a90 0x50a48d 0x50cd96 0x507d64

Loaded (5000) images of size (224, 224, 3).
tcmalloc: large alloc 6021120000 bytes == 0x7f76df1d0000 @  0x7f7c93fbc1e7 0x7f7c909b15e1 0x7f7c90a18420 0x7f7c90a0d7d5 0x7f7c90aa2497 0x50a635 0x50bfb4 0x507d64 0x50ae13 0x634c82 0x634d37 0x6384ef 0x639091 0x4b0d00 0x7f7c93bb9b97 0x5b250a
500
1000
1500
2000
2500
3000
3500
4000
4500
5000
done
Total number of depthmap images generated as of now : 40001 
time: 9min 45s


## output_5.zip : batches

### Batches : 1

In [76]:
!python test.py --input '/content/drive/My Drive/EVA4/S15A_RCNN_RGB/bg_fg_mask_jpgIms/output_5.zip' --start 0 --end 5000 -o '/content/drive/My Drive/EVA4/S15A_RCNN_RGB/denseDepthMaps_S15/DenseDepth/depthMapOutput' -rs 0 -re 10001 -b 50

Using TensorFlow backend.
Loading model...

Model loaded (nyu.h5).
tcmalloc: large alloc 6021120000 bytes == 0x7f538ac50000 @  0x7f55e707f1e7 0x7f55e3a745e1 0x7f55e3ad8c78 0x7f55e3ad8d93 0x7f55e3b76ea8 0x7f55e3b77704 0x7f55e3b77852 0x566d63 0x59fc4e 0x7f55e3ac44ed 0x50a2bf 0x50bfb4 0x507d64 0x509a90 0x50a48d 0x50cd96 0x507d64 0x588e5c 0x59fc4e 0x7f55e3ac44ed 0x50a2bf 0x50bfb4 0x507d64 0x509a90 0x50a48d 0x50cd96 0x507d64 0x509a90 0x50a48d 0x50cd96 0x507d64

Loaded (5000) images of size (224, 224, 3).
tcmalloc: large alloc 6021120000 bytes == 0x7f50331d0000 @  0x7f55e707f1e7 0x7f55e3a745e1 0x7f55e3adb420 0x7f55e3ad07d5 0x7f55e3b65497 0x50a635 0x50bfb4 0x507d64 0x50ae13 0x634c82 0x634d37 0x6384ef 0x639091 0x4b0d00 0x7f55e6c7cb97 0x5b250a
500
1000
1500
2000
2500
3000
3500
4000
4500
5000
done
time: 9min 34s


In [77]:
verifyCount()

Total number of depthmap images generated as of now : 45001 
time: 568 ms


### Batches : 2

In [78]:
!python test.py --input '/content/drive/My Drive/EVA4/S15A_RCNN_RGB/bg_fg_mask_jpgIms/output_5.zip' --start 0 --end 5000 -o '/content/drive/My Drive/EVA4/S15A_RCNN_RGB/denseDepthMaps_S15/DenseDepth/depthMapOutput' -rs 10000 -re 20001 -b 100
verifyCount()

Using TensorFlow backend.
Loading model...

Model loaded (nyu.h5).
tcmalloc: large alloc 6021120000 bytes == 0x7f2765950000 @  0x7f29c1de21e7 0x7f29be7d75e1 0x7f29be83bc78 0x7f29be83bd93 0x7f29be8d9ea8 0x7f29be8da704 0x7f29be8da852 0x566d63 0x59fc4e 0x7f29be8274ed 0x50a2bf 0x50bfb4 0x507d64 0x509a90 0x50a48d 0x50cd96 0x507d64 0x588e5c 0x59fc4e 0x7f29be8274ed 0x50a2bf 0x50bfb4 0x507d64 0x509a90 0x50a48d 0x50cd96 0x507d64 0x509a90 0x50a48d 0x50cd96 0x507d64

Loaded (5000) images of size (224, 224, 3).
tcmalloc: large alloc 6021120000 bytes == 0x7f23ff1d0000 @  0x7f29c1de21e7 0x7f29be7d75e1 0x7f29be83e420 0x7f29be8337d5 0x7f29be8c8497 0x50a635 0x50bfb4 0x507d64 0x50ae13 0x634c82 0x634d37 0x6384ef 0x639091 0x4b0d00 0x7f29c19dfb97 0x5b250a
500
1000
1500
2000
2500
3000
3500
4000
4500
5000
done
Total number of depthmap images generated as of now : 50001 
time: 9min 32s


### Batches : 3

In [81]:
!python test.py --input '/content/drive/My Drive/EVA4/S15A_RCNN_RGB/bg_fg_mask_jpgIms/output_5.zip' --start 0 --end 5000 -o '/content/drive/My Drive/EVA4/S15A_RCNN_RGB/denseDepthMaps_S15/DenseDepth/depthMapOutput' -rs 20000 -re 30001 -b 200
verifyCount()

Using TensorFlow backend.
Loading model...

Model loaded (nyu.h5).
tcmalloc: large alloc 6021120000 bytes == 0x7f423e690000 @  0x7f449aac71e7 0x7f44974bc5e1 0x7f4497520c78 0x7f4497520d93 0x7f44975beea8 0x7f44975bf704 0x7f44975bf852 0x566d63 0x59fc4e 0x7f449750c4ed 0x50a2bf 0x50bfb4 0x507d64 0x509a90 0x50a48d 0x50cd96 0x507d64 0x588e5c 0x59fc4e 0x7f449750c4ed 0x50a2bf 0x50bfb4 0x507d64 0x509a90 0x50a48d 0x50cd96 0x507d64 0x509a90 0x50a48d 0x50cd96 0x507d64

Loaded (5000) images of size (224, 224, 3).
At batch sz : 200
tcmalloc: large alloc 6021120000 bytes == 0x7f3d3f1d0000 @  0x7f449aac71e7 0x7f44974bc5e1 0x7f4497523420 0x7f44975187d5 0x7f44975ad497 0x50a635 0x50bfb4 0x507d64 0x50ae13 0x634c82 0x634d37 0x6384ef 0x639091 0x4b0d00 0x7f449a6c4b97 0x5b250a
500
1000
1500
2000
2500
3000
3500
4000
4500
5000
done
Total number of depthmap images generated as of now : 55001 
time: 9min 49s


### Batches : 4

In [83]:
!python test.py --input '/content/drive/My Drive/EVA4/S15A_RCNN_RGB/bg_fg_mask_jpgIms/output_5.zip' --start 0 --end 5000 -o '/content/drive/My Drive/EVA4/S15A_RCNN_RGB/denseDepthMaps_S15/DenseDepth/depthMapOutput' -rs 30000 -re 40001 -b 300
verifyCount()

Using TensorFlow backend.
Loading model...

Model loaded (nyu.h5).
tcmalloc: large alloc 6021120000 bytes == 0x7f742cad0000 @  0x7f7688eff1e7 0x7f76858f45e1 0x7f7685958c78 0x7f7685958d93 0x7f76859f6ea8 0x7f76859f7704 0x7f76859f7852 0x566d63 0x59fc4e 0x7f76859444ed 0x50a2bf 0x50bfb4 0x507d64 0x509a90 0x50a48d 0x50cd96 0x507d64 0x588e5c 0x59fc4e 0x7f76859444ed 0x50a2bf 0x50bfb4 0x507d64 0x509a90 0x50a48d 0x50cd96 0x507d64 0x509a90 0x50a48d 0x50cd96 0x507d64

Loaded (5000) images of size (224, 224, 3).
At batch sz : 300
tcmalloc: large alloc 6021120000 bytes == 0x7f6f2d1d0000 @  0x7f7688eff1e7 0x7f76858f45e1 0x7f768595b420 0x7f76859507d5 0x7f76859e5497 0x50a635 0x50bfb4 0x507d64 0x50ae13 0x634c82 0x634d37 0x6384ef 0x639091 0x4b0d00 0x7f7688afcb97 0x5b250a
500
1000
1500
2000
2500
3000
3500
4000
4500
5000
done
Total number of depthmap images generated as of now : 60001 
time: 10min 35s


### Batches : 5

In [87]:
!python test.py --input '/content/drive/My Drive/EVA4/S15A_RCNN_RGB/bg_fg_mask_jpgIms/output_5.zip' --start 0 --end 5000 -o '/content/drive/My Drive/EVA4/S15A_RCNN_RGB/denseDepthMaps_S15/DenseDepth/depthMapOutput' -rs 40000 -re 50001 -b 300
verifyCount()

Using TensorFlow backend.
Loading model...

Model loaded (nyu.h5).
tcmalloc: large alloc 6021120000 bytes == 0x7face5350000 @  0x7faf417791e7 0x7faf3e16e5e1 0x7faf3e1d2c78 0x7faf3e1d2d93 0x7faf3e270ea8 0x7faf3e271704 0x7faf3e271852 0x566d63 0x59fc4e 0x7faf3e1be4ed 0x50a2bf 0x50bfb4 0x507d64 0x509a90 0x50a48d 0x50cd96 0x507d64 0x588e5c 0x59fc4e 0x7faf3e1be4ed 0x50a2bf 0x50bfb4 0x507d64 0x509a90 0x50a48d 0x50cd96 0x507d64 0x509a90 0x50a48d 0x50cd96 0x507d64

Loaded (5000) images of size (224, 224, 3).
At batch sz : 300
utils predict Duration: 0:00:58.510465
tcmalloc: large alloc 6021120000 bytes == 0x7fa7e51d0000 @  0x7faf417791e7 0x7faf3e16e5e1 0x7faf3e1d5420 0x7faf3e1ca7d5 0x7faf3e25f497 0x50a635 0x50bfb4 0x507d64 0x50ae13 0x634c82 0x634d37 0x6384ef 0x639091 0x4b0d00 0x7faf41376b97 0x5b250a
500
1000
1500
2000
2500
3000
3500
4000
4500
5000
done
Total number of depthmap images generated as of now : 65001 
time: 10min 48s


### Batches : 6

In [91]:
!python test.py --input '/content/drive/My Drive/EVA4/S15A_RCNN_RGB/bg_fg_mask_jpgIms/output_5.zip' --start 0 --end 5000 -o '/content/drive/My Drive/EVA4/S15A_RCNN_RGB/denseDepthMaps_S15/DenseDepth/depthMapOutput' -rs 50000 -re 60001 -b 304
verifyCount()

Using TensorFlow backend.
Loading model...
Duration: 0:00:12.412128

Model loaded (nyu.h5).
tcmalloc: large alloc 6021120000 bytes == 0x7f1c81590000 @  0x7f1edd9cc1e7 0x7f1eda3c15e1 0x7f1eda425c78 0x7f1eda425d93 0x7f1eda4c3ea8 0x7f1eda4c4704 0x7f1eda4c4852 0x566d63 0x59fc4e 0x7f1eda4114ed 0x50a2bf 0x50bfb4 0x507d64 0x509a90 0x50a48d 0x50cd96 0x507d64 0x588e5c 0x59fc4e 0x7f1eda4114ed 0x50a2bf 0x50bfb4 0x507d64 0x509a90 0x50a48d 0x50cd96 0x507d64 0x509a90 0x50a48d 0x50cd96 0x507d64

Loaded (5000) images of size (224, 224, 3).
At batch sz : 304
utils predict Duration: 0:00:56.965842
tcmalloc: large alloc 6021120000 bytes == 0x7f17731d0000 @  0x7f1edd9cc1e7 0x7f1eda3c15e1 0x7f1eda428420 0x7f1eda41d7d5 0x7f1eda4b2497 0x50a635 0x50bfb4 0x507d64 0x50ae13 0x634c82 0x634d37 0x6384ef 0x639091 0x4b0d00 0x7f1edd5c9b97 0x5b250a
500
1000
1500
2000
2500
3000
3500
4000
4500
5000
done
Total number of depthmap images generated as of now : 70001 
time: 11min 18s


### Batches : 7

In [94]:
!python test.py --input '/content/drive/My Drive/EVA4/S15A_RCNN_RGB/bg_fg_mask_jpgIms/output_5.zip' --start 0 --end 5000 -o '/content/drive/My Drive/EVA4/S15A_RCNN_RGB/denseDepthMaps_S15/DenseDepth/depthMapOutput' -rs 60000 -re 70001 -b 304
verifyCount()

Using TensorFlow backend.
Loading model...
### Step-1: Time took to load model : 0:00:11.998196

Model loaded (nyu.h5).
tcmalloc: large alloc 6021120000 bytes == 0x7f20d3090000 @  0x7f232f4bc1e7 0x7f232beb15e1 0x7f232bf15c78 0x7f232bf15d93 0x7f232bfb3ea8 0x7f232bfb4704 0x7f232bfb4852 0x566d63 0x59fc4e 0x7f232bf014ed 0x50a2bf 0x50bfb4 0x507d64 0x509a90 0x50a48d 0x50cd96 0x507d64 0x588e5c 0x59fc4e 0x7f232bf014ed 0x50a2bf 0x50bfb4 0x507d64 0x509a90 0x50a48d 0x50cd96 0x507d64 0x509a90 0x50a48d 0x50cd96 0x507d64

Loaded (5000) images of size (224, 224, 3).
### Step-2: Time took to load Images : 0:00:57.230690
At batch sz : 304
utils predict Duration: 0:00:56.948307
### Step-3: Time took to compute results : 0:00:57.194098
tcmalloc: large alloc 6021120000 bytes == 0x7f1bd31d0000 @  0x7f232f4bc1e7 0x7f232beb15e1 0x7f232bf18420 0x7f232bf0d7d5 0x7f232bfa2497 0x50a635 0x50bfb4 0x507d64 0x50ae13 0x634c82 0x634d37 0x6384ef 0x639091 0x4b0d00 0x7f232f0b9b97 0x5b250a
500
1000
1500
2000
2500
3000
3500

### Batches : 8

In [95]:
!python test.py --input '/content/drive/My Drive/EVA4/S15A_RCNN_RGB/bg_fg_mask_jpgIms/output_5.zip' --start 0 --end 5000 -o '/content/drive/My Drive/EVA4/S15A_RCNN_RGB/denseDepthMaps_S15/DenseDepth/depthMapOutput' -rs 70000 -re 80001 -b 336
verifyCount()

Using TensorFlow backend.
Loading model...
### Step-1: Time took to load model : 0:00:12.042083

Model loaded (nyu.h5).
tcmalloc: large alloc 6021120000 bytes == 0x7fa70ed90000 @  0x7fa96b1db1e7 0x7fa967bd05e1 0x7fa967c34c78 0x7fa967c34d93 0x7fa967cd2ea8 0x7fa967cd3704 0x7fa967cd3852 0x566d63 0x59fc4e 0x7fa967c204ed 0x50a2bf 0x50bfb4 0x507d64 0x509a90 0x50a48d 0x50cd96 0x507d64 0x588e5c 0x59fc4e 0x7fa967c204ed 0x50a2bf 0x50bfb4 0x507d64 0x509a90 0x50a48d 0x50cd96 0x507d64 0x509a90 0x50a48d 0x50cd96 0x507d64

Loaded (5000) images of size (224, 224, 3).
### Step-2: Time took to load Images : 0:01:03.684425
At batch sz : 336
utils predict Duration: 0:01:01.454207
### Step-3: Time took to compute results : 0:01:01.666060
tcmalloc: large alloc 6021120000 bytes == 0x7fa0df1d0000 @  0x7fa96b1db1e7 0x7fa967bd05e1 0x7fa967c37420 0x7fa967c2c7d5 0x7fa967cc1497 0x50a635 0x50bfb4 0x507d64 0x50ae13 0x634c82 0x634d37 0x6384ef 0x639091 0x4b0d00 0x7fa96add8b97 0x5b250a
500
1000
1500
2000
2500
3000
3500

## output_6.zip : 
* RAM Usage is heiht : with batches with dataset = 16K instead of 10K (nd 8K instead of 5K)

### Batches : 1

In [100]:
!python test.py --input '/content/drive/My Drive/EVA4/S15A_RCNN_RGB/bg_fg_mask_jpgIms/output_6.zip' --start 0 --end 5000 -o '/content/drive/My Drive/EVA4/S15A_RCNN_RGB/denseDepthMaps_S15/DenseDepth/depthMapOutput' -rs 0 -re 10001 -b 336
verifyCount()

Using TensorFlow backend.
Loading model...
### Step-1: Time took to load model : 0:00:12.551899

Model loaded (nyu.h5).
tcmalloc: large alloc 6021120000 bytes == 0x7feafa190000 @  0x7fed565c31e7 0x7fed52fb85e1 0x7fed5301cc78 0x7fed5301cd93 0x7fed530baea8 0x7fed530bb704 0x7fed530bb852 0x566d63 0x59fc4e 0x7fed530084ed 0x50a2bf 0x50bfb4 0x507d64 0x509a90 0x50a48d 0x50cd96 0x507d64 0x588e5c 0x59fc4e 0x7fed530084ed 0x50a2bf 0x50bfb4 0x507d64 0x509a90 0x50a48d 0x50cd96 0x507d64 0x509a90 0x50a48d 0x50cd96 0x507d64

Loaded (5000) images of size (224, 224, 3).
### Step-2: Time took to load Images : 0:00:51.383947
At batch sz : 336
utils predict Duration: 0:01:01.562873
### Step-3: Time took to compute results : 0:01:01.779726
tcmalloc: large alloc 6021120000 bytes == 0x7fe4cb1d0000 @  0x7fed565c31e7 0x7fed52fb85e1 0x7fed5301f420 0x7fed530147d5 0x7fed530a9497 0x50a635 0x50bfb4 0x507d64 0x50ae13 0x634c82 0x634d37 0x6384ef 0x639091 0x4b0d00 0x7fed561c0b97 0x5b250a
500
1000
1500
2000
2500
3000
3500

### Batches : 2

In [101]:
!python test.py --input '/content/drive/My Drive/EVA4/S15A_RCNN_RGB/bg_fg_mask_jpgIms/output_6.zip' --start 0 --end 5000 -o '/content/drive/My Drive/EVA4/S15A_RCNN_RGB/denseDepthMaps_S15/DenseDepth/depthMapOutput' -rs 10000 -re 20001 -b 368
verifyCount()

Using TensorFlow backend.
Loading model...
### Step-1: Time took to load model : 0:00:12.967651

Model loaded (nyu.h5).
tcmalloc: large alloc 6021120000 bytes == 0x7f4c2e050000 @  0x7f4e8a4921e7 0x7f4e86e875e1 0x7f4e86eebc78 0x7f4e86eebd93 0x7f4e86f89ea8 0x7f4e86f8a704 0x7f4e86f8a852 0x566d63 0x59fc4e 0x7f4e86ed74ed 0x50a2bf 0x50bfb4 0x507d64 0x509a90 0x50a48d 0x50cd96 0x507d64 0x588e5c 0x59fc4e 0x7f4e86ed74ed 0x50a2bf 0x50bfb4 0x507d64 0x509a90 0x50a48d 0x50cd96 0x507d64 0x509a90 0x50a48d 0x50cd96 0x507d64

Loaded (5000) images of size (224, 224, 3).
### Step-2: Time took to load Images : 0:00:53.158642
At batch sz : 368
utils predict Duration: 0:00:59.986258
### Step-3: Time took to compute results : 0:01:00.210209
tcmalloc: large alloc 6021120000 bytes == 0x7f45f11d0000 @  0x7f4e8a4921e7 0x7f4e86e875e1 0x7f4e86eee420 0x7f4e86ee37d5 0x7f4e86f78497 0x50a635 0x50bfb4 0x507d64 0x50ae13 0x634c82 0x634d37 0x6384ef 0x639091 0x4b0d00 0x7f4e8a08fb97 0x5b250a
500
1000
1500
2000
2500
3000
3500

### Batches : 3

In [102]:
!python test.py --input '/content/drive/My Drive/EVA4/S15A_RCNN_RGB/bg_fg_mask_jpgIms/output_6.zip' --start 0 --end 5000 -o '/content/drive/My Drive/EVA4/S15A_RCNN_RGB/denseDepthMaps_S15/DenseDepth/depthMapOutput' -rs 20000 -re 30001 -b 368
verifyCount()

Using TensorFlow backend.
Loading model...
### Step-1: Time took to load model : 0:00:12.664964

Model loaded (nyu.h5).
tcmalloc: large alloc 6021120000 bytes == 0x7f5f19050000 @  0x7f61754461e7 0x7f6171e3b5e1 0x7f6171e9fc78 0x7f6171e9fd93 0x7f6171f3dea8 0x7f6171f3e704 0x7f6171f3e852 0x566d63 0x59fc4e 0x7f6171e8b4ed 0x50a2bf 0x50bfb4 0x507d64 0x509a90 0x50a48d 0x50cd96 0x507d64 0x588e5c 0x59fc4e 0x7f6171e8b4ed 0x50a2bf 0x50bfb4 0x507d64 0x509a90 0x50a48d 0x50cd96 0x507d64 0x509a90 0x50a48d 0x50cd96 0x507d64

Loaded (5000) images of size (224, 224, 3).
### Step-2: Time took to load Images : 0:00:53.385605
At batch sz : 368
utils predict Duration: 0:00:59.924398
### Step-3: Time took to compute results : 0:01:00.134921
tcmalloc: large alloc 6021120000 bytes == 0x7f58db1d0000 @  0x7f61754461e7 0x7f6171e3b5e1 0x7f6171ea2420 0x7f6171e977d5 0x7f6171f2c497 0x50a635 0x50bfb4 0x507d64 0x50ae13 0x634c82 0x634d37 0x6384ef 0x639091 0x4b0d00 0x7f6175043b97 0x5b250a
500
1000
1500
2000
2500
3000
3500

### Batches : 4

In [103]:
!python test.py --input '/content/drive/My Drive/EVA4/S15A_RCNN_RGB/bg_fg_mask_jpgIms/output_6.zip' --start 0 --end 5000 -o '/content/drive/My Drive/EVA4/S15A_RCNN_RGB/denseDepthMaps_S15/DenseDepth/depthMapOutput' -rs 30000 -re 40001 -b 368
verifyCount()

Using TensorFlow backend.
Loading model...
### Step-1: Time took to load model : 0:00:12.498628

Model loaded (nyu.h5).
tcmalloc: large alloc 6021120000 bytes == 0x7fa05e410000 @  0x7fa2ba8671e7 0x7fa2b725c5e1 0x7fa2b72c0c78 0x7fa2b72c0d93 0x7fa2b735eea8 0x7fa2b735f704 0x7fa2b735f852 0x566d63 0x59fc4e 0x7fa2b72ac4ed 0x50a2bf 0x50bfb4 0x507d64 0x509a90 0x50a48d 0x50cd96 0x507d64 0x588e5c 0x59fc4e 0x7fa2b72ac4ed 0x50a2bf 0x50bfb4 0x507d64 0x509a90 0x50a48d 0x50cd96 0x507d64 0x509a90 0x50a48d 0x50cd96 0x507d64

Loaded (5000) images of size (224, 224, 3).
### Step-2: Time took to load Images : 0:00:57.480193
At batch sz : 368
utils predict Duration: 0:00:59.976153
### Step-3: Time took to compute results : 0:01:00.198982
tcmalloc: large alloc 6021120000 bytes == 0x7f9a2f1d0000 @  0x7fa2ba8671e7 0x7fa2b725c5e1 0x7fa2b72c3420 0x7fa2b72b87d5 0x7fa2b734d497 0x50a635 0x50bfb4 0x507d64 0x50ae13 0x634c82 0x634d37 0x6384ef 0x639091 0x4b0d00 0x7fa2ba464b97 0x5b250a
500
1000
1500
2000
2500
3000
3500

### Batches : 5

In [112]:
!python test.py --input '/content/drive/My Drive/EVA4/S15A_RCNN_RGB/bg_fg_mask_jpgIms/output_6.zip' --start 0 --end 5000 -o '/content/drive/My Drive/EVA4/S15A_RCNN_RGB/denseDepthMaps_S15/DenseDepth/depthMapOutput' -rs 40000 -re 50001 -b 368
verifyCount()

Using TensorFlow backend.
Loading model...
### Step-1: Time took to load model : 0:00:12.404474

Model loaded (nyu.h5).
tcmalloc: large alloc 6021120000 bytes == 0x7fe943c10000 @  0x7feba8ca31e7 0x7feba56985e1 0x7feba56fcc78 0x7feba56fcd93 0x7feba579aea8 0x7feba579b704 0x7feba579b852 0x566d63 0x59fc4e 0x7feba56e84ed 0x50a2bf 0x50bfb4 0x507d64 0x509a90 0x50a48d 0x50cd96 0x507d64 0x588e5c 0x59fc4e 0x7feba56e84ed 0x50a2bf 0x50bfb4 0x507d64 0x509a90 0x50a48d 0x50cd96 0x507d64 0x509a90 0x50a48d 0x50cd96 0x507d64

Loaded (5000) images of size (224, 224, 3).
### Step-2: Time took to load Images : 0:00:56.091842
At batch sz : 368
utils predict Duration: 0:00:59.694877
### Step-3: Time took to compute results : 0:00:59.916553
tcmalloc: large alloc 6021120000 bytes == 0x7fe3131d0000 @  0x7feba8ca31e7 0x7feba56985e1 0x7feba56ff420 0x7feba56f47d5 0x7feba5789497 0x50a635 0x50bfb4 0x507d64 0x50ae13 0x634c82 0x634d37 0x6384ef 0x639091 0x4b0d00 0x7feba88a0b97 0x5b250a
500
1000
1500
2000
2500
3000
3500

### Batches : 6

In [115]:
!python test.py --input '/content/drive/My Drive/EVA4/S15A_RCNN_RGB/bg_fg_mask_jpgIms/output_6.zip' --start 0 --end 5000 -o '/content/drive/My Drive/EVA4/S15A_RCNN_RGB/denseDepthMaps_S15/DenseDepth/depthMapOutput' -rs 50000 -re 60001 -b 368
verifyCount()

Using TensorFlow backend.
Loading model...
### Step-1: Time took to load model : 0:00:12.613759

Model loaded (nyu.h5).
tcmalloc: large alloc 6021120000 bytes == 0x7f0365d50000 @  0x7f05cae0d1e7 0x7f05c78025e1 0x7f05c7866c78 0x7f05c7866d93 0x7f05c7904ea8 0x7f05c7905704 0x7f05c7905852 0x566d63 0x59fc4e 0x7f05c78524ed 0x50a2bf 0x50bfb4 0x507d64 0x509a90 0x50a48d 0x50cd96 0x507d64 0x588e5c 0x59fc4e 0x7f05c78524ed 0x50a2bf 0x50bfb4 0x507d64 0x509a90 0x50a48d 0x50cd96 0x507d64 0x509a90 0x50a48d 0x50cd96 0x507d64

Loaded (5000) images of size (224, 224, 3).
### Step-2: Time took to load Images : 0:00:58.505678
At batch sz : 368
utils predict Duration: 0:00:59.942965
### Step-3: Time took to compute results : 0:01:00.168242
tcmalloc: large alloc 6021120000 bytes == 0x7efd271d0000 @  0x7f05cae0d1e7 0x7f05c78025e1 0x7f05c7869420 0x7f05c785e7d5 0x7f05c78f3497 0x50a635 0x50bfb4 0x507d64 0x50ae13 0x634c82 0x634d37 0x6384ef 0x639091 0x4b0d00 0x7f05caa0ab97 0x5b250a
500
1000
1500
2000
2500
3000
3500

### Batches : 7

In [116]:
!python test.py --input '/content/drive/My Drive/EVA4/S15A_RCNN_RGB/bg_fg_mask_jpgIms/output_6.zip' --start 0 --end 5000 -o '/content/drive/My Drive/EVA4/S15A_RCNN_RGB/denseDepthMaps_S15/DenseDepth/depthMapOutput' -rs 60000 -re 70001 -b 368
verifyCount()

Using TensorFlow backend.
Loading model...
### Step-1: Time took to load model : 0:00:12.417356

Model loaded (nyu.h5).
tcmalloc: large alloc 6021120000 bytes == 0x7fae7f710000 @  0x7fb0e47a81e7 0x7fb0e119d5e1 0x7fb0e1201c78 0x7fb0e1201d93 0x7fb0e129fea8 0x7fb0e12a0704 0x7fb0e12a0852 0x566d63 0x59fc4e 0x7fb0e11ed4ed 0x50a2bf 0x50bfb4 0x507d64 0x509a90 0x50a48d 0x50cd96 0x507d64 0x588e5c 0x59fc4e 0x7fb0e11ed4ed 0x50a2bf 0x50bfb4 0x507d64 0x509a90 0x50a48d 0x50cd96 0x507d64 0x509a90 0x50a48d 0x50cd96 0x507d64

Loaded (5000) images of size (224, 224, 3).
### Step-2: Time took to load Images : 0:00:58.156599
At batch sz : 368
utils predict Duration: 0:01:00.205848
### Step-3: Time took to compute results : 0:01:00.425989
tcmalloc: large alloc 6021120000 bytes == 0x7fa8411d0000 @  0x7fb0e47a81e7 0x7fb0e119d5e1 0x7fb0e1204420 0x7fb0e11f97d5 0x7fb0e128e497 0x50a635 0x50bfb4 0x507d64 0x50ae13 0x634c82 0x634d37 0x6384ef 0x639091 0x4b0d00 0x7fb0e43a5b97 0x5b250a
500
1000
1500
2000
2500
3000
3500

### Batches : 8

In [117]:
!python test.py --input '/content/drive/My Drive/EVA4/S15A_RCNN_RGB/bg_fg_mask_jpgIms/output_6.zip' --start 0 --end 5000 -o '/content/drive/My Drive/EVA4/S15A_RCNN_RGB/denseDepthMaps_S15/DenseDepth/depthMapOutput' -rs 70000 -re 80001 -b 368
verifyCount()

Using TensorFlow backend.
Loading model...
### Step-1: Time took to load model : 0:00:16.249937

Model loaded (nyu.h5).
tcmalloc: large alloc 6021120000 bytes == 0x7fd0f85d0000 @  0x7fd35d67e1e7 0x7fd35a0735e1 0x7fd35a0d7c78 0x7fd35a0d7d93 0x7fd35a175ea8 0x7fd35a176704 0x7fd35a176852 0x566d63 0x59fc4e 0x7fd35a0c34ed 0x50a2bf 0x50bfb4 0x507d64 0x509a90 0x50a48d 0x50cd96 0x507d64 0x588e5c 0x59fc4e 0x7fd35a0c34ed 0x50a2bf 0x50bfb4 0x507d64 0x509a90 0x50a48d 0x50cd96 0x507d64 0x509a90 0x50a48d 0x50cd96 0x507d64

Loaded (5000) images of size (224, 224, 3).
### Step-2: Time took to load Images : 0:01:01.262772
At batch sz : 368
utils predict Duration: 0:01:00.235880
### Step-3: Time took to compute results : 0:01:00.460677
tcmalloc: large alloc 6021120000 bytes == 0x7fcabb1d0000 @  0x7fd35d67e1e7 0x7fd35a0735e1 0x7fd35a0da420 0x7fd35a0cf7d5 0x7fd35a164497 0x50a635 0x50bfb4 0x507d64 0x50ae13 0x634c82 0x634d37 0x6384ef 0x639091 0x4b0d00 0x7fd35d27bb97 0x5b250a
500
1000
1500
2000
2500
3000
3500

# Plot and generate test.png file 

In [0]:
#!pwd
#!ls
from matplotlib import pyplot as plt
from skimage import io

plt.figure(figsize=(40,40))
plt.imshow( io.imread('test.png') )